# 00 – Pose Estimation & Angle Export

This notebook processes all raw videos from `data/input/` using MediaPipe-based pose estimation, extracts joint angles per exercise type (using `ANGLE_CONFIG`), and saves outputs to `data/processed/`.

## Imports & Project Setup

In [1]:
# !pip install fastdtw
# !pip install mediapipe
# !pip install supervision

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation')

In [3]:
# ✅ Environment Setup
from utils.setup import setup_environment
base_path = setup_environment(mount_gdrive=True)

# ✅ Imports
import os, cv2, json, random
import numpy as np
import pandas as pd
from tqdm import tqdm
import supervision as sv
from supervision import KeyPoints
from pathlib import Path
from glob import glob
import mediapipe as mp
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from mediapipe.tasks import python as mp_tasks
from mediapipe.tasks.python import vision as mp_vision
from utils.config import COLORS as COLOR_HEXES, ANGLE_CONFIG, POSE_LANDMARKS, LABELS, FEATURE_CONFIG
from utils.config import EXERCISES, PROCESSED_DIR, INPUT_DIR, MODELS_DIR
from utils.pose_utils import (calculate_angle, export_angle_data, summarize_and_enrich_metadata,
                              overlay_joint_angles, get_angle_index, extract_rep_features)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Environment set. Project base path: /content/drive/MyDrive/pose-estimation-research


## Define Pose Estimation Setup

In [4]:
COLORS = [sv.Color.from_hex(color_hex=c) for c in COLOR_HEXES]
model_path = os.path.join(MODELS_DIR, "pose_landmarker_heavy.task")

# Set up MediaPipe Pose Landmarker in Video Mode
options_video = mp.tasks.vision.PoseLandmarkerOptions(
    base_options=mp.tasks.BaseOptions(model_asset_path=model_path),
    running_mode=mp.tasks.vision.RunningMode.VIDEO,
    num_poses=1,
    min_pose_detection_confidence=0.1,
    min_pose_presence_confidence=0.1,
    output_segmentation_masks=False
)

# Set up annotators
vertex_annotator = sv.VertexAnnotator(color=sv.Color.GREEN, radius=2)
edge_annotator = sv.EdgeAnnotator(color=sv.Color.GREEN, thickness=2)
vertex_label_annotator = sv.VertexLabelAnnotator(
    color=COLORS,
    text_color=sv.Color.BLACK,
    border_radius=2
)

PoseLandmarker = mp.tasks.vision.PoseLandmarker

## Utility Functions:
- Generate Human Readable Reports

In [5]:
def generate_human_readable_report(features: dict, exercise: str) -> str:
    report = f"📝 Movement Report for Exercise: {exercise.capitalize()}\n\n"
    score_tally = {
        "mobility": [],
        "tempo": [],
        "variability": [],
        "symmetry": [],
        "consistency": [],
    }

    for name, value in features.items():
        if isinstance(value, dict):  # scored features
            val = value["value"]
            score = value["score"]
            level = value["level"]

            if "symmetry" in name:
                joint = name.replace("_symmetry_std", "").replace("_angle", "").capitalize()
                report += f"🧍 {joint} Symmetry Std: {val:.2f} → {level} balance.\n"
                score_tally["symmetry"].append(score)

            elif "rom_dropoff" in name:
                joint = name.replace("_rom_dropoff", "").replace("_left", "").replace("_right", "").replace("_angle", "").capitalize()
                side = "Left" if "left" in name else "Right"
                report += f"📉 {joint} ROM Drop ({side}): {val:.1f}° → {level}.\n"
                score_tally["consistency"].append(score)

            elif "dtw_degradation" in name:
                joint = name.replace("_dtw_degradation", "").replace("_left", "").replace("_right", "").replace("_angle", "").capitalize()
                side = "Left" if "left" in name else "Right"
                report += f"🪢 {joint} DTW Degradation ({side}): {val:.1f} → {level}.\n"
                score_tally["consistency"].append(score)

        else:  # unstructured raw metrics
            if "rom" in name:
                side = "Left" if "left" in name else "Right"
                joint = name.replace("_rom", "").replace("_left", "").replace("_right", "").replace("_angle", "").capitalize()
                if value < 90:
                    status = "limited"
                    score = 1
                elif value < 120:
                    status = "moderate"
                    score = 2
                else:
                    status = "good"
                    score = 3
                report += f"🔄 {joint} ROM ({side}): {value:.1f}° → {status} mobility.\n"
                score_tally["mobility"].append(score)

            elif "velocity_mean" in name:
                side = "Left" if "left" in name else "Right"
                joint = name.replace("_velocity_mean", "").replace("_left", "").replace("_right", "").replace("_angle", "").capitalize()
                if abs(value) < 50:
                    status = "controlled"
                    score = 3
                elif abs(value) < 200:
                    status = "moderate"
                    score = 2
                else:
                    status = "explosive"
                    score = 1
                direction = "flexion" if value < 0 else "extension"
                report += f"⚡ {joint} Avg Angular Velocity ({side}): {value:.1f}°/s → {status} tempo ({direction}).\n"
                score_tally["tempo"].append(score)

            elif "velocity_max" in name:
                side = "Left" if "left" in name else "Right"
                joint = name.replace("_velocity_max", "").replace("_left", "").replace("_right", "").replace("_angle", "").capitalize()
                report += f"⚡ {joint} Max Velocity ({side}): {value:.1f}°/s → peak movement speed.\n"

            elif "velocity_std" in name:
                side = "Left" if "left" in name else "Right"
                joint = name.replace("_velocity_std", "").replace("_left", "").replace("_right", "").replace("_angle", "").capitalize()
                if value < 50:
                    variability = "consistent"
                    score = 3
                elif value < 150:
                    variability = "moderately variable"
                    score = 2
                else:
                    variability = "highly variable"
                    score = 1
                report += f"📉 {joint} Velocity Std Dev ({side}): {value:.1f}°/s → {variability} movement.\n"
                score_tally["variability"].append(score)

    # --- Summary scoring ---
    report += f"\n\n🔎 Summary Ratings:\n"
    for category, scores in score_tally.items():
        if scores:
            avg = np.mean(scores)
            level = "excellent" if avg >= 2.7 else "moderate" if avg >= 1.7 else "needs improvement"
            report += f"• {category.capitalize()}: {level} ({avg:.2f}/3)\n"

    return report

In [6]:
def score_metric(value, metric_type):
    if metric_type == "rom":
        if value >= 120:
            return {"score": 3, "level": "good"}
        elif value >= 90:
            return {"score": 2, "level": "moderate"}
        else:
            return {"score": 1, "level": "limited"}
    elif metric_type == "velocity_std":
        if value < 50:
            return {"score": 3, "level": "consistent"}
        elif value < 150:
            return {"score": 2, "level": "moderately variable"}
        else:
            return {"score": 1, "level": "highly variable"}
    elif metric_type == "symmetry_std":
        if value < 5:
            return {"score": 3, "level": "balanced"}
        elif value < 15:
            return {"score": 2, "level": "slightly asymmetric"}
        else:
            return {"score": 1, "level": "imbalanced"}
    elif metric_type == "dtw_degradation":
        if value < 1000:
            return {"score": 3, "level": "stable form"}
        elif value < 2000:
            return {"score": 2, "level": "moderate degradation"}
        else:
            return {"score": 1, "level": "severe degradation"}
    return {"score": 0, "level": "n/a"}

In [7]:
def extract_numeric_metrics(metrics_dict):
    result = {}
    for k, v in metrics_dict.items():
        if isinstance(v, (float, int)):
            result[k] = v
        elif isinstance(v, dict) and "value" in v:
            result[k] = v["value"]
    return result

## Define Feature Functions:
- Range of Motion
- Symmetry Score: TBI
- Angular Velocity
- Extract Features From Angles

In [8]:
def symmetry_score(left_series, right_series):
    """
    Compute a basic symmetry score between left and right joint angles.

    Parameters:
        left_series (pd.Series): Left side joint angle time series
        right_series (pd.Series): Right side joint angle time series

    Returns:
        float: Symmetry score ∈ [0, 1]; 1 = perfect symmetry

    Biomechanical Insight:
        - Detects muscle imbalances or coordination issues
        - Asymmetry may increase injury risk or reduce efficiency

    Application:
        - Running, pressing, bilateral exercises (e.g., squats, deadlifts)
    """
    diff = (left_series - right_series).abs()
    return 1 - diff.mean() / 180  # normalized to angle space

In [9]:
def compute_rom(series: pd.Series) -> float:
    return float(series.max() - series.min())

In [10]:
def compute_velocity(series: pd.Series, fps: float) -> pd.Series:
    dt = 1 / fps
    return series.diff() / dt

In [11]:
def extract_features_from_angles(df: pd.DataFrame, fps: float, exercise: str, config: dict) -> dict:
    """
    Extract ROM and angular velocity features from angle CSV.

    Args:
        df (pd.DataFrame): DataFrame with angle columns.
        fps (float): Video frame rate.
        exercise (str): Exercise name (e.g. "squat").
        config (dict): FEATURE_CONFIG dict.

    Returns:
        dict: Summary of extracted features.
    """
    summary = {}
    features = config[exercise]["features"]

    for feature_name, meta in features.items():
        base_name = feature_name.replace("_velocity", "")
        sides = meta.get("sides", ["left", "right"])
        feat_type = meta["type"]

        for side in sides:
            col = f"{base_name}_{side}"
            if col not in df.columns:
                continue

            if feat_type == "rom":
                summary[f"{col}_rom"] = compute_rom(df[col])

            elif feat_type == "velocity":
                vel_series = compute_velocity(df[col], fps)
                summary[f"{col}_velocity_mean"] = float(vel_series.mean())
                summary[f"{col}_velocity_max"] = float(vel_series.max())
                summary[f"{col}_velocity_std"] = float(vel_series.std())

    return summary


In [12]:
def compute_symmetry_std(df: pd.DataFrame, angle_prefix: str) -> float:
    """
    Computes the standard deviation between left and right joint angles.
    """
    left_col = f"{angle_prefix}_left"
    right_col = f"{angle_prefix}_right"

    if left_col in df.columns and right_col in df.columns:
        diff = df[left_col] - df[right_col]
        return float(diff.std())
    return np.nan

In [13]:
def compute_rom_dropoff(df: pd.DataFrame, col: str, window_size=30, stride=15) -> float:
    """
    Computes ROM drop-off: first rep ROM - last rep ROM
    """
    series = df[col].dropna().values
    roms = []
    for i in range(0, len(series) - window_size + 1, stride):
        window = series[i:i + window_size]
        roms.append(np.max(window) - np.min(window))
    if len(roms) >= 2:
        return float(roms[0] - roms[-1])
    return np.nan

In [14]:
def compute_dtw_degradation(df: pd.DataFrame, col: str, window_size=30, stride=15) -> float:
    """
    Compares all windows to the first using DTW and returns average DTW distance.
    Fixes input shape to (T, 1) for univariate DTW.
    """
    def ensure_scalar(x):
        if isinstance(x, (np.ndarray, list)):
            return float(np.array(x).flatten()[0])
        return float(x)

    series = df[col].dropna().apply(ensure_scalar).to_numpy()

    windows = [
        series[i:i + window_size]
        for i in range(0, len(series) - window_size + 1, stride)
    ]

    if len(windows) < 2:
        return np.nan

    baseline = windows[0]
    distances = []
    for idx, w in enumerate(windows[1:], start=1):
        if len(w) != len(baseline):
            continue
        try:
            # Reshape to (T, 1) so each timestep is a 1D vector
            d, _ = fastdtw(w.reshape(-1, 1), baseline.reshape(-1, 1), dist=euclidean)
            distances.append(d)
        except Exception as e:
            print(f"❌ Error in DTW comparison (window {idx}): {e}")

    return float(np.mean(distances)) if distances else np.nan

## Define Core Pose Estimation Extraction
- Pose Estimation Points
- Features: Range of Motion, Symmetry Scores, Angular Velocity
- Annotation: With Edges, With Edges & Labels

#### Return
- Dataframe [ body_part_angle_side, timestamp_ms, body_part_angle_velocity ]
- CSV Output
- Metadata JSON

In [ ]:
# def extract_pose_estimation_video(video_path, exercise, output_path=False, report_path="feature_report.json", annotate_with_angles=True, annotate_with_edges=True ,annotate_with_labels=False):
#     angle_definitions = ANGLE_CONFIG[exercise]["angles"]

#     # --- Video IO ---
#     cap = cv2.VideoCapture(video_path)
#     frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     writer = None

#     if fps == 0 or np.isnan(fps):
#       fps = 30.0

#     angle_data = []
#     timestamps = []

#     if output_path:
#       fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#       writer = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

#     with PoseLandmarker.create_from_options(options_video) as landmarker:
#         for frame_idx in tqdm(range(frame_count), desc=f"Processing {video_path} video"):
#             ret, frame = cap.read()
#             if not ret:
#                 break

#             # Time-based pose detection
#             timestamp_ms = int((frame_idx / fps) * 1000)
#             mp_frame = mp.Image(
#                 image_format=mp.ImageFormat.SRGB,
#                 data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             )
#             result = landmarker.detect_for_video(mp_frame, timestamp_ms)
#             keypoints = sv.KeyPoints.from_mediapipe(result, (frame_width, frame_height))
#             coords = keypoints.xy[0] if len(keypoints.xy) else None

#             if coords is None:
#                 frame_idx += 1
#                 continue

#             angle_row = {}
#             for side in ["left", "right"]:  # use angle_config["sides"] if passed dynamically
#               for angle_name, angle_meta in angle_definitions.items():
#                   try:
#                       j1, j2, j3 = angle_meta["joints"]
#                       key1 = f"{side.upper()}_{j1.upper()}"
#                       key2 = f"{side.upper()}_{j2.upper()}"
#                       key3 = f"{side.upper()}_{j3.upper()}"

#                       idx1 = POSE_LANDMARKS[key1]
#                       idx2 = POSE_LANDMARKS[key2]
#                       idx3 = POSE_LANDMARKS[key3]

#                       p1 = np.array(coords[idx1])
#                       p2 = np.array(coords[idx2])
#                       p3 = np.array(coords[idx3])

#                       angle_row[f"{angle_name}_{side}"] = calculate_angle(p1, p2, p3)
#                   except Exception:
#                       angle_row[f"{angle_name}_{side}"] = np.nan


#             annotated_frame = frame.copy()

#             angle_data.append(angle_row)
#             timestamps.append(timestamp_ms)

#             if writer:
#                 for i, (name, val) in enumerate(angle_row.items()):
#                   cv2.putText(frame, f"{name}: {val:.1f}", (10, 30 + i*20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1)
#                   if annotate_with_edges:
#                     annotated_frame = vertex_annotator.annotate(scene=annotated_frame, key_points=keypoints)
#                     annotated_frame = edge_annotator.annotate(scene=annotated_frame, key_points=keypoints)
#                     if annotate_with_labels:
#                       annotated_frame = vertex_label_annotator.annotate(scene=annotated_frame, key_points=keypoints, labels=LABELS)
#                 writer.write(annotated_frame)

#             frame_idx += 1

#     cap.release()
#     if writer:
#         writer.release()

#     df = pd.DataFrame(angle_data)
#     df["timestamp_ms"] = timestamps

#     # Save CSV next to annotated video/report
#     csv_path = report_path.replace("_features_metadata.json", "_angles.csv")
#     df.to_csv(csv_path, index=False)

#     # Feature extraction + human-readable report
#     fps=30
#     summary = extract_features_from_angles(df, fps, exercise, FEATURE_CONFIG)
#     interpretation = generate_human_readable_report(summary, exercise)

#     # Extend summary with degradation metrics
#     angle_cols = [col for col in df.columns if "_angle_" in col or col.endswith("_angle")]

#     for col in angle_cols:
#       base = col.rsplit("_", 1)[0]  # e.g., 'hip_angle'

#       # 1. Symmetry (std of left/right diff)
#       if f"{base}_left" in df.columns and f"{base}_right" in df.columns:
#           symmetry_val = compute_symmetry_std(df, base)
#           summary[f"{base}_symmetry_std"] = {
#               "value": symmetry_val,
#               **score_metric(symmetry_val, "symmetry_std")
#           }

#       # 2. ROM Drop-off (early vs late reps)
#       rom_drop = compute_rom_dropoff(df, col)
#       summary[f"{col}_rom_dropoff"] = {
#           "value": rom_drop,
#           **score_metric(rom_drop, "rom_dropoff")
#       }

#       # 3. DTW Degradation (form consistency over time)
#       dtw_score = compute_dtw_degradation(df, col)
#       summary[f"{col}_dtw_degradation"] = {
#           "value": dtw_score,
#           **score_metric(dtw_score, "dtw_degradation")
#       }


#     features_output = {
#         "metrics": summary,
#         "interpretation": interpretation
#     }

#     with open(report_path, "w") as f:
#         json.dump(features_output, f, indent=4)


In [15]:
def extract_pose_estimation_video(video_path, exercise, output_path=False, report_path="feature_report.json",
                                  annotate_with_angles=True, annotate_with_edges=True, annotate_with_labels=False):
    angle_definitions = ANGLE_CONFIG[exercise]["angles"]

    # --- Video IO ---
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    writer = None

    if fps == 0 or np.isnan(fps):
        fps = 30.0

    angle_data = []
    timestamps = []

    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    with PoseLandmarker.create_from_options(options_video) as landmarker:
        for frame_idx in tqdm(range(frame_count), desc=f"Processing {video_path} video"):
            ret, frame = cap.read()
            if not ret:
                break

            # Time-based pose detection
            timestamp_ms = int((frame_idx / fps) * 1000)
            mp_frame = mp.Image(
                image_format=mp.ImageFormat.SRGB,
                data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            )
            result = landmarker.detect_for_video(mp_frame, timestamp_ms)
            keypoints = sv.KeyPoints.from_mediapipe(result, (frame_width, frame_height))
            coords = keypoints.xy[0] if len(keypoints.xy) else None

            if coords is None:
                frame_idx += 1
                continue

            angle_row = {}
            for side in ["left", "right"]:
                for angle_name, angle_meta in angle_definitions.items():
                    try:
                        j1, j2, j3 = angle_meta["joints"]
                        key1 = f"{side.upper()}_{j1.upper()}"
                        key2 = f"{side.upper()}_{j2.upper()}"
                        key3 = f"{side.upper()}_{j3.upper()}"

                        idx1 = POSE_LANDMARKS[key1]
                        idx2 = POSE_LANDMARKS[key2]
                        idx3 = POSE_LANDMARKS[key3]

                        p1 = np.array(coords[idx1])
                        p2 = np.array(coords[idx2])
                        p3 = np.array(coords[idx3])

                        angle_row[f"{angle_name}_{side}"] = calculate_angle(p1, p2, p3)
                    except Exception:
                        angle_row[f"{angle_name}_{side}"] = np.nan

            annotated_frame = frame.copy()
            angle_data.append(angle_row)
            timestamps.append(timestamp_ms)

            if writer:
                for i, (name, val) in enumerate(angle_row.items()):
                    cv2.putText(frame, f"{name}: {val:.1f}", (10, 30 + i * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
                if annotate_with_edges:
                    annotated_frame = vertex_annotator.annotate(scene=annotated_frame, key_points=keypoints)
                    annotated_frame = edge_annotator.annotate(scene=annotated_frame, key_points=keypoints)
                if annotate_with_labels:
                    annotated_frame = vertex_label_annotator.annotate(scene=annotated_frame, key_points=keypoints, labels=LABELS)
                writer.write(annotated_frame)

            frame_idx += 1

    cap.release()
    if writer:
        writer.release()

    df = pd.DataFrame(angle_data)
    df["timestamp_ms"] = timestamps

    # Save raw CSV
    csv_path = report_path.replace("_features_metadata.json", "_angles.csv")
    df.to_csv(csv_path, index=False)

    # === Feature extraction and report ===
    summary = extract_features_from_angles(df, fps=30, exercise=exercise, config=FEATURE_CONFIG)
    interpretation = generate_human_readable_report(summary, exercise)

    # === Add extra features (symmetry, ROM drop-off, DTW degradation) ===
    angle_cols = [col for col in df.columns if "_angle_" in col or col.endswith("_angle")]
    for col in angle_cols:
        base = col.rsplit("_", 1)[0]
        if f"{base}_left" in df.columns and f"{base}_right" in df.columns:
            symmetry_val = compute_symmetry_std(df, base)
            summary[f"{base}_symmetry_std"] = {
                "value": symmetry_val,
                **score_metric(symmetry_val, "symmetry_std")
            }

        rom_drop = compute_rom_dropoff(df, col)
        summary[f"{col}_rom_dropoff"] = {
            "value": rom_drop,
            **score_metric(rom_drop, "rom_dropoff")
        }

        dtw_score = compute_dtw_degradation(df, col)
        summary[f"{col}_dtw_degradation"] = {
            "value": dtw_score,
            **score_metric(dtw_score, "dtw_degradation")
        }

    # === Save report ===
    features_output = {
        "metrics": summary,
        "interpretation": interpretation
    }

    with open(report_path, "w") as f:
        json.dump(features_output, f, indent=4)

    # === Save enriched CSV with repeated video-level features per row ===
    video_feature_vector = extract_numeric_metrics(summary)
    video_feature_df = pd.DataFrame([video_feature_vector] * len(df))
    df_featured = pd.concat([df.reset_index(drop=True), video_feature_df.reset_index(drop=True)], axis=1)

    csv_path_featured = csv_path.replace("_angles.csv", "_angles_featured.csv")
    df_featured.to_csv(csv_path_featured, index=False)
    print(f"✅ Saved enriched angle + feature CSV: {csv_path_featured}")


## Pose Estimation Extraction Single Case

In [ ]:
# exercise = "squat"
# video_path = "/content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation/data/input/4921644-hd_1066_1920_25fps.mp4"

# extract_pose_estimation_video(video_path, exercise, output_path=False)

## Batch Pose Estimation Extraction
- Input directory
- Output directory
- Exercise: ["squat", "juggling", "bench_press", "pull_ups", "lunges"]

In [16]:
def process_all_videos_for_exercises(
    input_dir: str,
    output_dir: str,
    exercises: list,
    video_ext: str = ".mp4"
):
    """
    Batch process all videos by exercise using extract_pose_estimation_video.

    Args:
        input_dir (str): Root folder containing input videos organized per exercise.
                         e.g., data/input/squat/, data/input/bench_press/
        output_dir (str): Where to save annotated videos + features.json
        exercises (list): List of exercise names (must match ANGLE_CONFIG keys).
        video_ext (str): Video file extension to filter by.
    """
    for exercise in exercises:
        print(f"\n📂 Processing exercise: {exercise}")
        exercise_input_dir = os.path.join(input_dir, exercise)
        exercise_output_dir = os.path.join(output_dir, exercise)
        os.makedirs(exercise_output_dir, exist_ok=True)

        video_paths = glob(os.path.join(exercise_input_dir, f"*{video_ext}"))

        for video_path in video_paths:
          video_name = os.path.splitext(os.path.basename(video_path))[0]
          print(f"▶️  {video_name}")

          video_output_dir = os.path.join(exercise_output_dir, video_name)
          os.makedirs(video_output_dir, exist_ok=True)

          annotated_video_path = os.path.join(video_output_dir, f"{video_name}_annotated.mp4")
          report_path = os.path.join(video_output_dir, f"{video_name}_features_metadata.json")

          extract_pose_estimation_video(
              video_path=video_path,
              exercise=exercise,
              output_path=annotated_video_path,
              report_path=report_path
          )

In [18]:
EXERCISES = ["squat", "bench_press", "pull_ups", "lunges"]

process_all_videos_for_exercises(
    input_dir=INPUT_DIR,
    output_dir=PROCESSED_DIR,
    exercises=EXERCISES
)


📂 Processing exercise: squat
▶️  v_BodyWeightSquats_g16_c04


Processing /content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation/data/input/squat/v_BodyWeightSquats_g16_c04.mp4 video: 100%|██████████| 77/77 [00:11<00:00,  6.80it/s]


✅ Saved enriched angle + feature CSV: /content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation/data/processed/squat/v_BodyWeightSquats_g16_c04/v_BodyWeightSquats_g16_c04_angles_featured.csv
▶️  v_BodyWeightSquats_g07_c02


Processing /content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation/data/input/squat/v_BodyWeightSquats_g07_c02.mp4 video: 100%|██████████| 103/103 [00:14<00:00,  7.16it/s]


✅ Saved enriched angle + feature CSV: /content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation/data/processed/squat/v_BodyWeightSquats_g07_c02/v_BodyWeightSquats_g07_c02_angles_featured.csv
▶️  v_BodyWeightSquats_g21_c01


...

✅ Saved enriched angle + feature CSV: /content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation/data/processed/lunges/v_Lunges_g23_c03/v_Lunges_g23_c03_angles_featured.csv
▶️  v_Lunges_g15_c01


Processing /content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation/data/input/lunges/v_Lunges_g15_c01.mp4 video: 100%|██████████| 117/117 [00:16<00:00,  7.10it/s]


✅ Saved enriched angle + feature CSV: /content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation/data/processed/lunges/v_Lunges_g15_c01/v_Lunges_g15_c01_angles_featured.csv
▶️  v_Lunges_g05_c01


Processing /content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation/data/input/lunges/v_Lunges_g05_c01.mp4 video: 100%|██████████| 239/239 [00:33<00:00,  7.04it/s]


✅ Saved enriched angle + feature CSV: /content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation/data/processed/lunges/v_Lunges_g05_c01/v_Lunges_g05_c01_angles_featured.csv
